In [ ]:
import statsmodels.api as sm
import pandas as pd
import re,string
import nltk
from patsy import dmatrices
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import warnings
%pylab inline
import numpy as np
from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances
import spacy


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Populating the interactive namespace from numpy and matplotlib


# PwC Topic Modeling

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
from google.colab import files
uploaded1 = files.upload()

Saving PwC_Reviews.csv to PwC_Reviews.csv


In [ ]:
pwc_df = pd.read_csv('PwC_Reviews.csv', encoding="iso-8859-1")
pwc_df = pwc_df.rename(columns = {'ï»¿Title' : 'Title'})
pwc_df.head()

,Title,Date Written,Rating,Current/Former,Job Title,Location,Recommendation?,Outlook,Main Text,Pros,Cons
0,Amazing place to grow and develop,26.Sep.21,5,Former,Senior Management Consultant,NaN,Recommend,Business Outlook,Share,Focused a lot of early career development. Ama...,"Long hours, but honestly, not deal-breaking."
1,It's luck of the draw,27.Sep.21,3,Current,International Tax Associate,NaN,Recommend,Business Outlook,Share,"The people, you will gain a ton of experience ...",Although the hours are flexible they are still...
2,Lots of politics,28.Sep.21,4,Former,SAP Consultant,NaN,Recommend,Business Outlook,Share,Opportunity to work with lots of bright talent...,Very negative experience with a manager with z...
3,Its a massive ship,30.Sep.21,3,Former,International Tax Associate,NaN,Recommend,Business Outlook,Share,"They are trying to change culture, and you can...",They are still a business and you are replacea...
4,PwC life,21.Sep.21,4,Current,Risk and Regulatory Advisory Senior Associate,NaN,Recommend,Business Outlook,Share,Good pay and benefits Career opportunities are...,You work a lot. The firm preaches work life ba...


In [ ]:
pwc_df.isnull().sum()

Title                21
Date Written          0
Rating                0
Current/Former        0
Job Title             0
Location           1990
Recommendation?       0
Outlook               0
Main Text             0
Pros                  0
Cons                  0
dtype: int64

In [ ]:
pwc_df.shape

(1990, 11)

In [ ]:
import nltk
nltk.download('stopwords')
stop = set(stopwords.words('english'))
punc = string.punctuation    

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
pros = pd.DataFrame(pwc_df['Pros'], index= pwc_df.index)
cons = pd.DataFrame(pwc_df['Cons'],  index= pwc_df.index)

In [ ]:
pros.dropna(axis=0, inplace=True)
cons.dropna(axis=0, inplace=True)

In [ ]:
def clean_tokenize(s):
        s = re.sub(r'[^\w\s]', '',s.lower())
        return([word for word in word_tokenize(s) if word not in stop if word not in punc])
def get_lemma(word):
        lemma = wn.morphy(word)
        if lemma is None:
            return word
        else:
            return lemma
            
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)
    
def lemmatize(my_tokens):
    """Function to enable data preparation for topic modeling using LDA, 
    includes removing stop words, lemmatization, tokenization"""
    my_tokens = [get_lemma(token) for token in my_tokens]
    return my_tokens

def dataprep(text_data_df):
    pros = pd.DataFrame(text_data_df['Pros'], index= text_data_df.index)
    cons = pd.DataFrame(text_data_df['Cons'],  index= text_data_df.index)
    pros.dropna(axis=0, inplace=True)
    cons.dropna(axis=0, inplace=True)
    stop = set(stopwords.words('english'))
    punc = string.punctuation
    
    pros['pros_tokens'] =pros.Pros.map(clean_tokenize)
    cons['cons_tokens'] =cons.Cons.map(clean_tokenize)
    #my_tokens = [get_lemma(token) for token in my_tokens]
    pros['pros_lemma_tokens'] = pros['pros_tokens'].apply(lemmatize)
    #pros['pros_lemma_tokens'] = [get_lemma(token) for token in pros['pros_tokens']]
    cons['cons_lemma_tokens'] = cons['cons_tokens'].apply(lemmatize)
    return pros, cons

In [ ]:
import nltk
nltk.download('punkt')
pros['pros_tokens'] =pros.Pros.map(clean_tokenize)
cons['cons_tokens'] =cons.Cons.map(clean_tokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
pros.head()

,Pros,pros_tokens
0,Focused a lot of early career development. Ama...,"[focused, lot, early, career, development, ama..."
1,"The people, you will gain a ton of experience ...","[people, gain, ton, experience, fast, rate, fl..."
2,Opportunity to work with lots of bright talent...,"[opportunity, work, lots, bright, talented, pe..."
3,"They are trying to change culture, and you can...","[trying, change, culture, find, people, care]"
4,Good pay and benefits Career opportunities are...,"[good, pay, benefits, career, opportunities, p..."


In [ ]:
# In Gensim, the corpus contains the word id and its frequency in every document.
from gensim import corpora
dictionary = corpora.Dictionary(pros['pros_tokens']) # Create an object of corpora.Dictionary() 
corpus = [dictionary.doc2bow(text) for text in pros['pros_tokens']] # Now pass these tokenised sentences to dictionary.doc2bow() object
corpus[:5] # the result is Bag of Word corpus
# The below output shows that the word with id=0 appears once in the first document (because we have got (0,1) in the output) and so on.

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1)],
 [(14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1)],
 [(25, 1),
  (28, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1)],
 [(25, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1)],
 [(1, 1),
  (25, 1),
  (28, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1)]]

In [ ]:
# The above output is somehow not possible for humans to read. 
# We can also convert these ids to words but for this we need our dictionary to do the conversion
id_words = [[(dictionary[id], count) for id, count in line] for line in corpus]
print(id_words)

[[('amazing', 1), ('career', 1), ('development', 1), ('early', 1), ('focused', 1), ('grow', 1), ('lot', 1), ('place', 1), ('problem', 1), ('quantitative', 1), ('skills', 1), ('solving', 1), ('strategic', 1), ('thinking', 1)], [('challenges', 1), ('constantly', 1), ('environment', 1), ('experience', 1), ('fast', 1), ('flexible', 1), ('gain', 1), ('home', 1), ('hours', 1), ('hybrid', 1), ('job', 1), ('people', 1), ('rate', 1), ('ton', 1), ('work', 1), ('working', 1)], [('people', 1), ('work', 1), ('bright', 1), ('firm', 1), ('general', 1), ('lots', 1), ('opportunity', 1), ('prestige', 1), ('talented', 1)], [('people', 1), ('care', 1), ('change', 1), ('culture', 1), ('find', 1), ('trying', 1)], [('career', 1), ('people', 1), ('work', 1), ('benefits', 1), ('good', 1), ('opportunities', 1), ('pay', 1), ('plentiful', 1), ('progress', 1), ('well', 1)], [('amazing', 1), ('home', 1), ('people', 1), ('work', 1), ('benefits', 1), ('allows', 1), ('covid', 1), ('decent', 2), ('donât', 1), ('even', 

### <font color = 'green'>About Pickle...</font>
* The pickle module implements binary protocols for serializing and de-serializing a Python object structure - also called marshalling or flattening. “Pickling” is the process whereby a Python object hierarchy is converted into a byte stream, and “unpickling” is the inverse operation, whereby a byte stream is converted back into an object hierarchy.

* Pickle is very useful for when you're working with machine learning algorithms, where you want to save them to be able to make new predictions at a later time, without having to rewrite everything or train the model all over again.

* Open the file for writing, simply use the <b><i>open()</i></b> function. The <u>first argument</u> should be the name of your file. The <u>second argument</u> is <b><i>'wb'</i></b>. The w means that you'll be writing to the file, and <b><i>b</i></b> refers to binary mode. 

In [ ]:
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

### Now LDA... But what does LDA do?
* LDA’s approach to topic modeling is it considers each document as a collection of topics in a certain proportion. And each topic as a collection of keywords, again, in a certain proportion.
* Once you provide the algorithm with the number of topics, all it does it to rearrange the topics distribution within the documents and keywords distribution within the topics to obtain a good composition of topic-keywords distribution.
* A topic is nothing but a collection of dominant keywords that are typical representatives. Just by looking at the keywords, you can identify what the topic is all about.

#### Using LDA with Gensim package:
* <b><i>alpha</i></b> and <b><i>eta</i></b> are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior.
* <b><i>chunksize</i></b> is the number of documents to be used in each training chunk. <b><i>update_every</i></b> determines how often the model parameters should be updated and <b><i>passes</i></b> is the total number of training passes.
* The below LDA model is built with 5 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.

<u><i>source: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction</i></u>

In [ ]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.091*"great" + 0.048*"work" + 0.042*"people" + 0.038*"good"')
(1, '0.038*"work" + 0.035*"people" + 0.025*"culture" + 0.021*"flexible"')
(2, '0.070*"good" + 0.034*"people" + 0.030*"great" + 0.024*"benefits"')
(3, '0.047*"name" + 0.037*"work" + 0.027*"balance" + 0.025*"great"')
(4, '0.040*"learn" + 0.031*"lot" + 0.022*"opportunity" + 0.021*"work"')


In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(cons['cons_tokens'])
corpus = [dictionary.doc2bow(text) for text in cons['cons_tokens']]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [ ]:
print(cons['cons_tokens'])

0                   [long, hours, honestly, dealbreaking]
1       [although, hours, flexible, still, ridiculousl...
2       [negative, experience, manager, zero, people, ...
3       [still, business, replaceable, burnt, matter, ...
4       [work, lot, firm, preaches, work, life, balanc...
                              ...                        
1985    [long, busy, season, hour, stressed, first, year]
1986    [lots, travel, formal, mentors, hit, miss, lac...
1987    [poor, management, discipline, noncompetitive,...
1988     [work, life, balance, great, finish, cpa, enter]
1989       [poor, work, life, balance, poor, 401k, match]
Name: cons_tokens, Length: 1990, dtype: object


In [ ]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.075*"work" + 0.069*"balance" + 0.052*"life" + 0.020*"worklife"')
(1, '0.105*"hours" + 0.075*"long" + 0.041*"busy" + 0.036*"work"')
(2, '0.024*"hard" + 0.021*"work" + 0.013*"bad" + 0.011*"company"')
(3, '0.016*"pay" + 0.016*"big" + 0.012*"work" + 0.009*"better"')
(4, '0.020*"name" + 0.017*"work" + 0.016*"cons" + 0.013*"people"')


#### How to interpret this?
* Topic 0 is a represented as '0.032*"flexibility" + 0.025*"growth" + 0.023*"opportunities" + 0.019*"opportunity".
* It means the top 4 keywords that contribute to this topic are: 'flexibility', 'growth', ‘opportunities’ and 'opportunity' and the weight of ‘flexibility’ on topic 0 is 0.032.
* The weights reflect how important a keyword is to that topic.

### Compute Model Perplexity and Coherence Score
* Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is. In my experience, topic coherence score, in particular, has been more helpful.

In [ ]:
from gensim.models import CoherenceModel
# Compute Perplexity
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=pros['pros_tokens'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

NameError: ignored

### Visualize the topics-keywords
* Now that the LDA model is built, the next step is to examine the produced topics and the associated keywords. There is no better tool than pyLDAvis package’s interactive chart and is designed to work well with jupyter notebooks.


In [ ]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(ldamodel, corpus, dictionary)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.139340  0.034249       1        1  34.055674
0      0.051626 -0.104709       2        1  20.369633
1     -0.011529  0.091434       3        1  18.338592
2     -0.092250  0.027819       4        1  14.854710
4     -0.087187 -0.048792       5        1  12.381391, topic_info=             Term        Freq       Total Category  logprob  loglift
94          great  683.000000  683.000000  Default  30.0000  30.0000
43           good  430.000000  430.000000  Default  29.0000  29.0000
97       learning  153.000000  153.000000  Default  28.0000  28.0000
44  opportunities  262.000000  262.000000  Default  27.0000  27.0000
28           work  488.000000  488.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
55           nice    8.712335   69.270998   Topic5  -5.3578   0.0157
2     development    8.247933   54.398175   Topic5  -5.4126   0.2026
77        balance    8.318920   68.536762   Topic5  -5.4040  -0.0199
90           team    8.407447   95.133770   Topic5  -5.3934  -0.3372
44  opportunities    8.475646  262.886112   Topic5  -5.3854  -1.3455

[373 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
373       3  0.710529      1
373       5  0.142106      1
229       1  0.075811      2
229       3  0.682300      2
229       4  0.151622      2
...     ...       ...    ...
259       1  0.122872  young
259       2  0.081914  young
259       3  0.245743  young
259       4  0.368615  young
259       5  0.163829  young

[671 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 2, 3, 5])

* Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.

* A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.

* A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.
Alright, if you move the cursor over one of the bubbles, the words and bars on the right-hand side will update. These words are the salient keywords that form the selected topic.